# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic


In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (115 kB/s)
Reading package lis

In [ ]:
# Install java
# NB: Spark requires Java's runtime. It is one of the dependencies of Spark.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#Get spark 
!wget https://downloads.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop2.7.tgz

--2022-08-13 19:13:56--  https://downloads.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272846416 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop2.7.tgz.2’

spark-3.2.2-bin-had 100%[===================>] 260.21M  80.0MB/s    in 3.4s    

2022-08-13 19:14:00 (76.0 MB/s) - ‘spark-3.2.2-bin-hadoop2.7.tgz.2’ saved [272846416/272846416]



In [ ]:
#Decompress spark
!tar xf spark-3.2.2-bin-hadoop2.7.tgz    #tar command unarchives the folder

#Install python package to help with system paths
!pip install -q findspark         #python package helps spark connect to local runtime

In [ ]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop2.7"

In [ ]:
# Add pyspark to sys.path using findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd 
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

In [ ]:
spark

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [ ]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [ ]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set? 

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [ ]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [ ]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [ ]:
joined_df.groupby('name').agg({"price": ["sum"]}) 

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [ ]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [ ]:
# save to the filesystem to prevent another load
# ! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

In [ ]:
! wget "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv"

--2022-08-13 19:14:18--  https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv.2’

rows.csv.2              [        <=>         ] 391.42M  2.64MB/s    in 3m 25s  

2022-08-13 19:17:43 (1.91 MB/s) - ‘rows.csv.2’ saved [410439552]



In [ ]:
# ! curl -o rows.csv

In [ ]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

1917681


We'll take a random sample at 20% of the original data

In [ ]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [ ]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1', 
                       'BOROUGH', 'VEHICLE TYPE CODE 1', 
                       'NUMBER OF PERSONS INJURED']]

In [ ]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
797894,09/13/2017,Unspecified,NaN,Sedan,0.0
461101,03/23/2019,Following Too Closely,MANHATTAN,Station Wagon/Sport Utility Vehicle,0.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [ ]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out 
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/content/spark-3.2.2-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


StructType(List(StructField(CRASH DATE,StringType,true),StructField(CONTRIBUTING FACTOR VEHICLE 1,StringType,true),StructField(BOROUGH,StringType,true),StructField(VEHICLE TYPE CODE 1,StringType,true),StructField(NUMBER OF PERSONS INJURED,StringType,true)))


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [ ]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

CRASH DATE
01/21/2014    256
11/15/2018    202
01/07/2017    194
12/15/2017    194
02/03/2014    193
Name: CRASH DATE, dtype: int64

In [ ]:
### Spark?
sdf_small.show()

+----------+-----------------------------+-------------+--------------------+-------------------------+
|CRASH DATE|CONTRIBUTING FACTOR VEHICLE 1|      BOROUGH| VEHICLE TYPE CODE 1|NUMBER OF PERSONS INJURED|
+----------+-----------------------------+-------------+--------------------+-------------------------+
|09/13/2017|                  Unspecified|             |               Sedan|                      0.0|
|03/23/2019|         Following Too Clo...|    MANHATTAN|Station Wagon/Spo...|                      0.0|
|07/24/2015|                  Unspecified|       QUEENS|   PASSENGER VEHICLE|                      0.0|
|01/30/2019|         Driver Inattentio...|     BROOKLYN|               Sedan|                      0.0|
|07/15/2014|          Driver Inexperience|        BRONX|SPORT UTILITY / S...|                      0.0|
|11/30/2018|         Driver Inattentio...|             |               Sedan|                      0.0|
|05/03/2014|              Fatigued/Drowsy|             |      LI

In [ ]:
from pyspark.sql.functions import desc

In [ ]:
df = sdf_small.groupBy("CRASH DATE").count().sort(desc("count"))  #alternatively instead of grouping by count we can use .alias("sum of crash date") within the count function
df.show(5)

+----------+-----+
|CRASH DATE|count|
+----------+-----+
|01/21/2014|  256|
|11/15/2018|  202|
|01/07/2017|  194|
|12/15/2017|  194|
|02/03/2014|  193|
+----------+-----+
only showing top 5 rows



# Question 2

> _Where do most crashes occur?_

In [ ]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         83897
QUEENS           71074
MANHATTAN        60108
BRONX            38494
STATEN ISLAND    11098
Name: BOROUGH, dtype: int64

In [ ]:
## Spark?
df2 = sdf_small.groupBy("BOROUGH").count().sort(desc("count"))  
df2.show()


+-------------+------+
|      BOROUGH| count|
+-------------+------+
|             |118865|
|     BROOKLYN| 83897|
|       QUEENS| 71074|
|    MANHATTAN| 60108|
|        BRONX| 38494|
|STATEN ISLAND| 11098|
+-------------+------+



 # Question 3
 
 > What is the most common cause of accident in 'QUEENS'

In [ ]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       25758
Driver Inattention/Distraction    14987
Failure to Yield Right-of-Way      5957
Backing Unsafely                   3566
Following Too Closely              2541
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [ ]:
## Spark?
from pyspark.sql.functions import col
df3_ = sdf_small.filter(sdf_small.BOROUGH == "QUEENS")
df3 = df3_.groupBy("CONTRIBUTING FACTOR VEHICLE 1").count().sort(desc("count"))  
df3.show(5)



+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|25758|
|         Driver Inattentio...|14987|
|         Failure to Yield ...| 5957|
|             Backing Unsafely| 3566|
|         Following Too Clo...| 2541|
+-----------------------------+-----+
only showing top 5 rows



# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [ ]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

VEHICLE TYPE CODE 1  BOROUGH  
12 passage           BROOKLYN     5.0
amb                  MANHATTAN    4.0
PICKUP               BRONX        4.0
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [ ]:
## Spark?
from pyspark.sql.functions import sum,avg,max, col


In [ ]:
## Spark
df4 = sdf_small.select('BOROUGH', 'VEHICLE TYPE CODE 1','NUMBER OF PERSONS INJURED')
df_ = df4.groupBy('VEHICLE TYPE CODE 1', 'BOROUGH')\
.agg(f.avg('NUMBER OF PERSONS INJURED').alias('MEAN NUMBER OF PERSONS INJURED'))\
.orderBy('MEAN NUMBER OF PERSONS INJURED', ascending=False)
df_.show(5)

+-------------------+---------+------------------------------+
|VEHICLE TYPE CODE 1|  BOROUGH|MEAN NUMBER OF PERSONS INJURED|
+-------------------+---------+------------------------------+
|         12 passage| BROOKLYN|                           5.0|
|              BACKH|   QUEENS|                           4.0|
|                amb|MANHATTAN|                           4.0|
|                suv|   QUEENS|                           4.0|
|              PASSE|         |                           4.0|
+-------------------+---------+------------------------------+
only showing top 5 rows

